In [25]:
# imports
import cv2 as cv
import numpy as np
import time
import glob
from prettytable import PrettyTable

# video_file = "project"
video_file = "challenge"

DEBUG_MODE = True

# Time Measurement

In [26]:
time_measurements = {}

def start_time_measurement(eventName):
    """
    Use this function to start a time measurement for a specific event.
    A end_time_measurement() call with the same name must be called before the next start_time_measurement() call.
    """
    add = False
    if eventName not in time_measurements:
        time_measurements[eventName] = {
            "name": eventName,
            "start": [],
            "end": [],
            "count": 0
        }
        add = True
    elif len(time_measurements[eventName]["start"]) > len(time_measurements[eventName]["end"]):
        print(f"Time measure error: Event '{eventName}' not finished before reassignment!")
    else:
       add = True
    
    #start measurement as late as possible
    if add:
        time_measurements[eventName]["start"].append(time.perf_counter_ns())

def end_time_measurement(eventName):
    """
    Use this function to end a time measurement for a specific event.
    A time measurement with the same name must be started with start_time_measurement() before it can be ended.
    """
    #end measurement as fast as possible
    temp_time = time.perf_counter_ns()
    if eventName not in time_measurements:
        print(f"Time measure error: Event '{eventName}' not defined!")
    elif len(time_measurements[eventName]["end"]) >= len(time_measurements[eventName]["start"]):
        print(f"Time measure error: Event '{eventName}' not started before reassignment!")
    else:
        time_measurements[eventName]["end"].append(temp_time)
        time_measurements[eventName]["count"] += 1
    
def analyse_time_measurements():
    """
    Analyse time measurements and print them in a table.
    """
    time_measurements_table = PrettyTable(["Name", "Avg. [ms]", "Min. [ms]", "Max. [ms]", "Occurrences [compl.]"])
    time_measurements_table.align["Name"] = "l"
    time_measurements_table.align["Avg. [ms]"] = "r"
    time_measurements_table.align["Min. [ms]"] = "r"
    time_measurements_table.align["Max. [ms]"] = "r"
    time_measurements_table.align["Occurrences [compl.]"] = "r"
    for key, event in time_measurements.items():
        timings = []
        if len(event["start"]) != len(event["end"]):
            print(f"Time measure error: Event '{key}' has different amounts of values for start and end times!")
        else:
            #exclude 0 values
            for i in range(len(event["start"])):
                timing = (event["end"][i] - event["start"][i])
                if timing >= 0:
                    timing = timing / (1000 * 1000) #convert from ns to ms
                    timings.append(timing)

            event["min"] = min(timings) 
            event["max"] = max(timings)
            event["avg"] = sum(timings) / len(event["start"])

            time_measurements_table.add_row([key, '{0:.2f}'.format(event["avg"]), '{0:.2f}'.format(event["min"]), '{0:.2f}'.format(event["max"]), event["count"]])
    print(time_measurements_table)

# Kamerakalibrierung

In [27]:
# size of chessboard, minimum error with (7, 6), but there were severe artifacts at the borders (see error calculation at the end)
chessboard_x, chessboard_y = 9, 6

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboard_y * chessboard_x, 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_x, 0:chessboard_y].T.reshape(-1, 2)
# Arrays to store object points and image points from all the images.
object_points = []  # 3d point in real world space
image_points = []  # 2d points in image plane.

# use all calibration images
images = glob.glob("./img/Udacity/calib/*.jpg")
for i, fname in enumerate(images):
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (chessboard_x, chessboard_y), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        object_points.append(objp)
        corners2 = cv.cornerSubPix(
            gray, corners, (11, 11), (-1, -1), criteria
        )  # improve accuracy of corners
        image_points.append(corners)

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(
    object_points, image_points, gray.shape[::-1], None, None
)

image_height, image_width = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (image_width, image_height), 1, (image_width, image_height))


# ------- define functions for image processing -------
def undistort_image(img):
    img_undistorted = cv.undistort(img, mtx, dist, None, newcameramtx)
    # crop the image
    x, y, w, h = roi
    return img_undistorted[y : y + h, x : x + w]

# ~50% faster than undistort_image()
def undistort_image_remap(img):
    h, w = img.shape[:2]
    
    map_x, map_y = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w, h), 5)
    dst = cv.remap(img, map_x, map_y, cv.INTER_LINEAR)
    # crop the image
    x, y, w, h = roi
    return dst[y : y + h, x : x + w]

# Perspektivtransformation

Mögliche Performance-verbesserung:

- Bild nach warp verkleinern, da ein großteil des Bildes aus wenigen Pixeln entsteht -> nicht gemacht weil probleme beim rückwarpen
  


In [28]:
# udacity images
if video_file == "harder_challenge":
    src_udacity = np.float32([[20, 628], [191+100, 404], [1200, 628], [1021-100, 404]])
    dst_udacity = np.float32([[150, 720], [150, 10], [1000, 720], [1000, 10]])
else:
    src_udacity = np.float32([[191, 628], [531, 404], [1021, 628], [681, 404]])
    dst_udacity = np.float32([[150, 720], [150, 10], [1000, 720], [1000, 10]])

M_warp = cv.getPerspectiveTransform(src_udacity, dst_udacity)
M_rewarp = cv.getPerspectiveTransform(dst_udacity, src_udacity)


def warp_image_udacity(img):
    image = cv.warpPerspective(img, M_warp, (img.shape[1], img.shape[0]))
    if DEBUG_MODE:
        #Draw a red circle with zero radius and -1 for filled circle
        for src in src_udacity:
            image2 = cv.circle(img, np.int32(src), radius=0, color=(0, 0, 255), thickness=5)
        cv.imshow("Transformed", image2)
    return image

# todo deprecated in main branch
def rewarp_image_udacity(img):
    img = cv.warpPerspective(img, M_rewarp, (img.shape[1], img.shape[0]), cv.WARP_INVERSE_MAP)
    return img

def rewarp_points_udacity(points):
    """
    Rewarp points from warped image coordinates to original image coordinates.
    
    Args:
        points (np.array): float32 array 
        
    @return: points in original image coordinates
    """
    return cv.perspectiveTransform(points, M_rewarp)


# Sliding Windows

In [29]:
number_windows = 50
margin = 50
def sliding_windows(frame, window_width=200, minimum_whites=30):
    """_summary_

    Args:
        frame (image): input frame with masked lane lines
        window_width (int, optional): width of windows. Defaults to 200.
        minimum_whites (int, optional): todo. Defaults to 30.

    Returns:
        lefts, rights (np.array): returns left and right points as 2d array of sliding window centers
    """
    # Histogram for image
    hist = np.sum(frame[frame.shape[0]//2:, :], axis=0)
        
    # Take peaks from left and right side of histogram for starting points and add half margin
    mid_point_x = np.int32(hist.shape[0] // 2)
    left_x_start = np.argmax(hist[:mid_point_x]) - window_width // 2
    right_x_start = np.argmax(hist[mid_point_x:]) + mid_point_x + window_width // 2
    # Window height based on number of windows
    window_height = np.int32(frame.shape[0] // number_windows)
    
    # Calc points that are not zero in images
    nonzero = frame.nonzero()
    nonzero_y = np.array(nonzero[0])
    nonzero_x = np.array(nonzero[1])
    
    # Initialize current positions for windows
    left_x_current = left_x_start
    right_x_current = right_x_start

    # Initialize values to be returned -> centers of windows
    lefts_good = np.empty((0,2), dtype=np.int32)
    rights_good = np.empty((0,2), dtype=np.int32)

    # Go through every window
    for window in range(number_windows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = frame.shape[0] - (window + 1) * window_height
        win_y_high = frame.shape[0] - window*window_height
        y_mid = (win_y_low + win_y_high) // 2
        
        # Calculate boundaries of the window
        win_xleft_low = left_x_current - window_width  
        win_xleft_high = left_x_current + window_width  
        win_xright_low =  right_x_current - window_width 
        win_xright_high = right_x_current + window_width  
        
        # Identify the pixels that are not zero within window
        left_inds = ((nonzero_y >= win_y_low ) & (nonzero_y < win_y_high) & (nonzero_x >= win_xleft_low) & (nonzero_x < win_xleft_high)).nonzero()[0]
        right_inds = ((nonzero_y >= win_y_low ) & (nonzero_y < win_y_high) & (nonzero_x >= win_xright_low) & (nonzero_x < win_xright_high)).nonzero()[0]
        
        # If more than minimum pixels are found -> recenter next window
        if len(left_inds) > minimum_whites:
            left_x_current = np.int32(np.mean(nonzero_x[left_inds]))
            lefts_good = np.concatenate((lefts_good, [[left_x_current, y_mid]]))
            if DEBUG_MODE:
                cv.rectangle(frame, (left_x_current - margin, win_y_low),(left_x_current + margin, win_y_high),(255, 255, 255), 2)
        if len(right_inds) > minimum_whites:
            right_x_current = np.int32(np.mean(nonzero_x[right_inds]))
            rights_good = np.concatenate((rights_good, [[right_x_current, y_mid]]))
            if DEBUG_MODE:
                cv.rectangle(frame, (right_x_current - margin, win_y_low),(right_x_current + margin, win_y_high),(255, 255, 255), 2)

    return lefts_good, rights_good

# Polynomes
calculation from center points of the sliding windows

and drawing points on image

In [30]:
#  calculate polynomial from rewarped points
def calculate_polynomial_points(lefts, rights):
    """
    calculate polynomial from rewarped points and then return list of points on the polynomial
    @return: array of points for left and right polynomial
    """
    # check if there are enough points
    if len(lefts) < 2 or len(rights) < 2:
        return None, None
    
    # calculate polynomial from rewarped points
    left_polynom_values = np.polyfit(lefts[:,1], lefts[:,0], 2)
    right_polynom_values = np.polyfit(rights[:,1], rights[:,0], 2)

    # 750 as x length of polynom
    x_axis = np.linspace(0, 750, 750) 

    # calculate y values for left and right line
    left_line_y = left_polynom_values[0]*x_axis**2 + left_polynom_values[1]*x_axis + left_polynom_values[2]
    right_line_y = right_polynom_values[0]*x_axis**2 + right_polynom_values[1]*x_axis + right_polynom_values[2]

    # array of points for left and right line from x and y values
    left_pts = np.array([np.transpose(np.vstack([left_line_y, x_axis]))])
    right_pts = np.array([np.transpose(np.vstack([right_line_y, x_axis]))])

    return left_pts, right_pts

In [31]:
def drawRecOnFrame(frame, left_pts, right_pts):
    """
    Rewarp points to original image coordinates and draw rectangle on frame.
    
    input: array of points from left and right lane marking in warped image
    e.g.:
      [[ 281   39]
      [ 971  163]
      [ 958  101]]
    """
    if len(left_pts) + len(right_pts) > 3:
      borderPoints = np.concatenate((np.flip(left_pts, axis=0), right_pts))
      borderPointsRewarped = rewarp_points_udacity(np.array([borderPoints], dtype=np.float32))
      borderPointsRewarpedInt = borderPointsRewarped.astype(int)
      # draw final polygon in frame
      cv.drawContours(frame, borderPointsRewarpedInt, -1, (0,255,0), -1)

# Masking

In [32]:
# todo deprecated
def applyMasks(frame):
    """
    Apply masks to frame and return frame with only lane marking.
    """
    ## convert to hsv
    hls_frame = cv.cvtColor(frame, cv.COLOR_BGR2HLS)

    ## mask for white
    white_mask = cv.inRange(hls_frame, (0, 200, 0), (255, 255,255))

    ## mask for yellow
    # yellow_mask = cv.inRange(hls_frame, (20,90,200), (26, 255, 255))
    yellow_mask = cv.inRange(hls_frame, (10,0,100), (40, 255, 255))

    ## final mask and masked
    mask = cv.bitwise_or(white_mask, yellow_mask)
    frame = cv.bitwise_and(frame,frame, mask=mask)

    return frame

In [33]:
def lane_detection(frame, white_lower, white_upper, yellow_lower, yellow_upper):
    """_summary_

    Args:
        frame (_type_): _description_
        white_lower (_type_): _description_
        white_upper (_type_): _description_
        yellow_lower (_type_): _description_
        yellow_upper (_type_): _description_

    Returns:
        _type_: _description_
    """
    frame = cv.GaussianBlur(frame, (5, 5), 0)
    frame_hls = cv.cvtColor(frame, cv.COLOR_BGR2HLS)

    white_mask = cv.inRange(frame_hls, white_lower, white_upper)
    white_mask[:, 0:200] = 0

    frame_lab = cv.cvtColor(frame, cv.COLOR_BGR2LAB)
    yellow_mask = cv.inRange(frame_lab, yellow_lower, yellow_upper)
    yellow_mask[:, 1000:] = 0
    combined_mask = cv.bitwise_or(white_mask, yellow_mask)
    
    if DEBUG_MODE: # needs frame at this point
        frame_luv = cv.cvtColor(frame, cv.COLOR_BGR2LUV)
        greenImg = np.zeros(frame.shape, frame.dtype)
        redImg = np.zeros(frame.shape, frame.dtype)

    kernel = cv.getStructuringElement(cv.MORPH_RECT, (13, 13))
    lanes_yellow = cv.morphologyEx(frame_lab[:, :, 2], cv.MORPH_TOPHAT, kernel)

    ret, lanes_yellow = cv.threshold(lanes_yellow, thresh=2, maxval=255, type=cv.THRESH_BINARY)
    lanes_yellow = cv.morphologyEx(lanes_yellow, cv.MORPH_OPEN, np.array([[0,1,0],[1,1,1],[0,1,0]], 'uint8'), iterations=4)

    combined_mask = cv.bitwise_or(combined_mask, lanes_yellow)

    combined_mask = cv.morphologyEx(combined_mask, cv.MORPH_OPEN, np.array([[0,1,0],[1,1,1],[0,1,0]], 'uint8'), iterations=2)
    frame = cv.bitwise_and(frame, frame, mask=combined_mask)
    frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    
    if DEBUG_MODE: # show multiple debug images
        cv.imshow("LUV", frame_luv[:,:,2])
        
        redImg[:,:] = (0, 0, 255)
        redMask = cv.bitwise_and(redImg, redImg, mask=yellow_mask)

        greenImg[:,:] = (0, 255, 0)
        greenMask = cv.bitwise_and(greenImg, greenImg, mask=white_mask)
        masks = redMask + greenMask
        cv.imshow("Masks", masks)
        
        lanes = cv.morphologyEx(frame_hls[:, :, 1], cv.MORPH_BLACKHAT, kernel)
        ret, lanes = cv.threshold(lanes, thresh=10, maxval=255, type=cv.THRESH_BINARY)
        cv.imshow("BLACK_HAT_LANES", lanes)

        ret, canny = cv.threshold(frame_lab[:,:,2], thresh=140, maxval=255, type=cv.THRESH_BINARY)
        half = canny.shape[1]//2
        new_canny = canny[100:, :half] 
        cv.imshow("LAB_Canny", new_canny)
    
        cv.imshow("LAB", frame_lab[:,:,2])
        cv.imshow("LAB_combined", cv.bitwise_and(canny, lanes_yellow))
        cv.imshow("HLS", frame_hls)
        
        cv.imshow("TOP_HAT_LANES_YELLOW", lanes_yellow)
        cv.imshow("Final", combined_mask)
    return frame

In [34]:
def correct_Brightness(frame):
    frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    brightness = frame_hsv[...,2].mean()
    brightness_upper_limit = 150
    brightness_lower_limit = 85
    if brightness > brightness_upper_limit:
        h, s, v = cv.split(frame_hsv)
        
        lim = round(0 + brightness - brightness_upper_limit)
        v[v < lim] = 0
        v[v >= lim] -= lim

        final_hsv = cv.merge((h, s, v))
        frame = cv.cvtColor(final_hsv, cv.COLOR_HSV2BGR)
    
    elif brightness < brightness_lower_limit:
        h, s, v = cv.split(frame_hsv)
        
        lim = round(255 + brightness_lower_limit - brightness)
        v[v > lim] = 255
        v[v <= lim] += lim

        final_hsv = cv.merge((h, s, v))
        frame = cv.cvtColor(final_hsv, cv.COLOR_HSV2BGR)

    frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    brightness = frame_hsv[...,2].mean()

    return frame, brightness

In [35]:
def on_change(value):
    global next
    next = True

# Main


In [36]:
# Open video file
capture = cv.VideoCapture("./img/Udacity/" + video_file + "_video.mp4")

# Check if camera opened successfully
if capture.isOpened() == False:
    print("Error opening video stream or file")

# Start timer for fps counter
start_timer = time.time() - 0.01
frame_count = -1

frames = []
next = True

while capture.isOpened():
    ret, frame = capture.read()

    # Check if there is another frame
    if frame is None:
        break

    frames.append(frame)

position = 0
window = cv.namedWindow("Options", cv.WINDOW_NORMAL)
cv.createTrackbar('Frame', 'Options', 0, len(frames), on_change)

cv.createTrackbar('white_lower_1', 'Options', 0, 255, on_change)
cv.createTrackbar('white_lower_2', 'Options', 0, 255, on_change)
cv.createTrackbar('white_lower_3', 'Options', 0, 255, on_change)

cv.createTrackbar('white_upper_1', 'Options', 0, 255, on_change)
cv.createTrackbar('white_upper_2', 'Options', 0, 255, on_change)
cv.createTrackbar('white_upper_3', 'Options', 0, 255, on_change)

cv.createTrackbar('yellow_lower_1', 'Options', 0, 255, on_change)
cv.createTrackbar('yellow_lower_2', 'Options', 0, 255, on_change)
cv.createTrackbar('yellow_lower_3', 'Options', 0, 255, on_change)

cv.createTrackbar('yellow_upper_1', 'Options', 0, 255, on_change)
cv.createTrackbar('yellow_upper_2', 'Options', 0, 255, on_change)
cv.createTrackbar('yellow_upper_3', 'Options', 0, 255, on_change)

yellow_lower = np.array([150, 100, 140])
yellow_upper = np.array([255, 140, 200])

white_lower = np.array([0, 180, 0])
white_upper = np.array([254, 254, 254])

cv.setTrackbarPos('white_lower_1', 'Options', white_lower[0])
cv.setTrackbarPos('white_lower_2', 'Options', white_lower[1])
cv.setTrackbarPos('white_lower_3', 'Options', white_lower[2])

cv.setTrackbarPos('white_upper_1', 'Options', white_upper[0]) 
cv.setTrackbarPos('white_upper_2', 'Options', white_upper[1])
cv.setTrackbarPos('white_upper_3', 'Options', white_upper[2])

cv.setTrackbarPos('yellow_lower_1', 'Options', yellow_lower[0])
cv.setTrackbarPos('yellow_lower_2', 'Options', yellow_lower[1])
cv.setTrackbarPos('yellow_lower_3', 'Options', yellow_lower[2])

cv.setTrackbarPos('yellow_upper_1', 'Options', yellow_upper[0])
cv.setTrackbarPos('yellow_upper_2', 'Options', yellow_upper[1])
cv.setTrackbarPos('yellow_upper_3', 'Options', yellow_upper[2])

# Read every frame
while position < len(frames):

    frame = frames[position]

    # Check if there is another frame
    if frame is None:
        break

    # Calculate Frame rate
    if next:

        white_lower[0] = cv.getTrackbarPos('white_lower_1', 'Options')
        white_lower[1] = cv.getTrackbarPos('white_lower_2', 'Options')
        white_lower[2] = cv.getTrackbarPos('white_lower_3', 'Options')

        white_upper[0] = cv.getTrackbarPos('white_upper_1', 'Options')
        white_upper[1] = cv.getTrackbarPos('white_upper_2', 'Options')
        white_upper[2] = cv.getTrackbarPos('white_upper_3', 'Options')

        yellow_lower[0] = cv.getTrackbarPos('yellow_lower_1', 'Options')
        yellow_lower[1] = cv.getTrackbarPos('yellow_lower_2', 'Options')
        yellow_lower[2] = cv.getTrackbarPos('yellow_lower_3', 'Options')

        yellow_upper[0] = cv.getTrackbarPos('yellow_upper_1', 'Options')
        yellow_upper[1] = cv.getTrackbarPos('yellow_upper_2', 'Options')
        yellow_upper[2] = cv.getTrackbarPos('yellow_upper_3', 'Options')


        frame_count += 1
        elapsed_time = time.time() - start_timer
        frame_rate = frame_count / elapsed_time
        
        start_time_measurement("frame")
        
        # ----------- Preprocessing ---------------
        start_time_measurement("Preprocessing")
        frame = undistort_image_remap(frame)
        frame_undistorted = frame.copy()
        frame = warp_image_udacity(frame)
        end_time_measurement("Preprocessing")


        img = frame
        height = img.shape[0]
        width = img.shape[1]

        width_cutoff = width // 2

        left1 = img[:, :width_cutoff]
        right1 = img[:, width_cutoff:]

        img = cv.rotate(left1, cv.ROTATE_90_CLOCKWISE)

        height = img.shape[0]
        width = img.shape[1]
        width_cutoff = width // 2

        l1 = img[:, :width_cutoff]
        l2 = img[:, width_cutoff:]
        l1 = cv.rotate(l1, cv.ROTATE_90_COUNTERCLOCKWISE)
        l2 = cv.rotate(l2, cv.ROTATE_90_COUNTERCLOCKWISE)

        img = cv.rotate(right1, cv.ROTATE_90_CLOCKWISE)

        height = img.shape[0]
        width = img.shape[1]
        width_cutoff = width // 2

        r1 = img[:, :width_cutoff]
        r2 = img[:, width_cutoff:]
        r1 = cv.rotate(r1, cv.ROTATE_90_COUNTERCLOCKWISE)
        r2 = cv.rotate(r2, cv.ROTATE_90_COUNTERCLOCKWISE)

        cv.imshow("one_horisont_1.jpg", l1)
        cv.imshow("one_horisont_2.jpg", l2)
        cv.imshow("second_vhorisont_1.jpg", r1)
        cv.imshow("second_horisont_2.jpg", r2)
        
        # ---------  Masking ---------------------
        start_time_measurement("masking")
        # frame = applyMasks(frame)
        l1, brightness = correct_Brightness(l1)
        grayscale_frame_l1 = lane_detection(l1, white_lower, white_upper, yellow_lower, yellow_upper)

        l2, brightness = correct_Brightness(l2)
        grayscale_frame_l2 = lane_detection(l2, white_lower, white_upper, yellow_lower, yellow_upper)

        r1, brightness = correct_Brightness(r1)
        grayscale_frame_r1 = lane_detection(r1, white_lower, white_upper, yellow_lower, yellow_upper)

        r2, brightness = correct_Brightness(r2)
        grayscale_frame_r2 = lane_detection(r2, white_lower, white_upper, yellow_lower, yellow_upper)
        end_time_measurement("masking")


        numpy_vertical_l = np.vstack((grayscale_frame_l2, grayscale_frame_l1))
        numpy_vertical_r = np.vstack((grayscale_frame_r2, grayscale_frame_r1))
        grayscale_frame = np.hstack((numpy_vertical_l, numpy_vertical_r))

        cv.imshow("Test", grayscale_frame)
        #---------- Sliding Windows ----------
        start_time_measurement("sliding windows")
        # Convert to grayscale for sliding windows
        lefts, rights = sliding_windows(grayscale_frame, minimum_whites=margin)
        end_time_measurement("sliding windows")


        #---------- Calculate polynomial ----------
        start_time_measurement("polynomial calculation")
        left_pts, right_pts = calculate_polynomial_points(lefts, rights)
        end_time_measurement("polynomial calculation")

        # -------- draw plane from all sliding windows ------------
        start_time_measurement("draw plane")
        if left_pts is not None and right_pts is not None:
            drawRecOnFrame(frame_undistorted, left_pts[0], right_pts[0])
        end_time_measurement("draw plane")

        # -------- Add frame rate to video ------------
        cv.putText(frame_undistorted, "FPS: " + str(round(frame_rate)), (0, 25),
                    cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv.LINE_AA,)
        cv.putText(frame_undistorted, "Frame: " + str(frame_count), (0, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv.LINE_AA,)
        cv.putText(frame_undistorted, "Brightness: " + str(np.round(brightness)), (0, 75), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv.LINE_AA,)
        cv.imshow("Frame", frame_undistorted)

        end_time_measurement("frame")
        next = False
        
    pressedKey = cv.waitKey(1) & 0xFF
    if pressedKey == ord('q'):
        break
    elif pressedKey == ord('w'):
        position += 1
        cv.setTrackbarPos('Frame', 'Options', position)
        next = True
    elif pressedKey == ord('s') and position > 0:
        position -= 1
        cv.setTrackbarPos('Frame', 'Options', position)
        next = True
    elif cv.getTrackbarPos('Frame', 'Options') != position:
        position = cv.getTrackbarPos('Frame', 'Options')
        next = True

# When everything done, release the video capture object
capture.release()
cv.destroyAllWindows()

# create time measurement analysis and print out results
analyse_time_measurements()

C:\Users\Eddi\AppData\Local\Temp\ipykernel_21224\2470665765.py:176: RankWarning: Polyfit may be poorly conditioned
  left_pts, right_pts = calculate_polynomial_points(lefts, rights)


+------------------------+-----------+-----------+-----------+----------------------+
| Name                   | Avg. [ms] | Min. [ms] | Max. [ms] | Occurrences [compl.] |
+------------------------+-----------+-----------+-----------+----------------------+
| frame                  |     52.34 |     41.99 |    307.51 |                   34 |
| Preprocessing          |      5.25 |      4.48 |     21.01 |                   34 |
| masking                |     33.61 |     26.92 |    181.13 |                   34 |
| sliding windows        |      6.50 |      5.19 |     20.58 |                   34 |
| polynomial calculation |      0.25 |      0.23 |      0.58 |                   34 |
| draw plane             |      0.19 |      0.17 |      0.22 |                   34 |
+------------------------+-----------+-----------+-----------+----------------------+
